In [2]:
import tensorflow as tf
import numpy as np
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

X_train1 = X_train[y_train < 5]
y_train1 = y_train[y_train < 5]
X_valid1 = X_valid[y_valid < 5]
y_valid1 = y_valid[y_valid < 5]
X_test1 = X_test[y_test < 5]
y_test1 = y_test[y_test < 5]


X_train2 = X_train[y_train >= 5]
y_train2 = y_train[y_train>=5]
X_test2 = X_test[y_test >= 5]
y_test2 = y_test[y_test >=5]

Now build a simple tf neural network consisting of 5 layers that trains on categories 1-5.

What must we do to build this?
* Specify the model scope - building the layers and output
* Specify the optimizer - adam optimizer, the loss function, what needs to be minimized, etc.
* Specify the training scope, the process of training the data
* Specify the batch size, number of epochs
* Specify the placeholders for the data

In [20]:
import random
tf.reset_default_graph()
n_inputs = 28*28
n_outputs = 5

learning_rate = 0.01
he_init = tf.variance_scaling_initializer()
X_op,y_op = X_train1,y_train1

#Build the base model. Make it simple, dont use a function

X = tf.placeholder(tf.float32, shape=(None,n_inputs),name="X")
y = tf.placeholder(tf.int32, shape=(None),name="y")


def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

n_epochs = 1000
batch_size = 50

    
n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10


with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("error"):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,logits=logits)
    loss = tf.reduce_mean(cross_entropy,name="loss")

with tf.name_scope("evaluate"):
    correct = tf.nn.in_top_k(logits,y,1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init) 
    for epoch in range(n_epochs):           
        for X_batch, y_batch in shuffle_batch(X_op, y_op, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        loss_val = sess.run(loss,feed_dict={X:X_batch, y:y_batch})
        if epoch % int(n_epochs*(0.05))==0:
            print("Epoch: ",epoch," Loss: ",loss_val," Accuracy: ",acc)

Epoch:  0  Loss:  0.023098188  Accuracy:  1.0
Epoch:  50  Loss:  0.008209795  Accuracy:  1.0
Epoch:  100  Loss:  0.83128136  Accuracy:  0.66
Epoch:  150  Loss:  0.92094964  Accuracy:  0.58
Epoch:  200  Loss:  1.024651  Accuracy:  0.4
Epoch:  250  Loss:  0.846916  Accuracy:  0.56
Epoch:  300  Loss:  1.0061889  Accuracy:  0.42
Epoch:  350  Loss:  0.8780724  Accuracy:  0.54
Epoch:  400  Loss:  0.95714754  Accuracy:  0.58
Epoch:  450  Loss:  1.0327398  Accuracy:  0.48
Epoch:  500  Loss:  0.9751179  Accuracy:  0.5
Epoch:  550  Loss:  1.1201516  Accuracy:  0.44
Epoch:  600  Loss:  1.0318096  Accuracy:  0.6
Epoch:  650  Loss:  0.9122088  Accuracy:  0.6
Epoch:  700  Loss:  0.80729216  Accuracy:  0.6
Epoch:  750  Loss:  0.90291905  Accuracy:  0.52
Epoch:  800  Loss:  1.025439  Accuracy:  0.46
Epoch:  850  Loss:  0.94736564  Accuracy:  0.62
Epoch:  900  Loss:  1.0016252  Accuracy:  0.56
Epoch:  950  Loss:  0.99115133  Accuracy:  0.52


In [ ]:
    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(X_op,y_op, batch_size)
            sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
        loss_val,summary_str = sess.run([loss,loss_summary], feed_dict={X:X_op,y:y_op})
        file_writer.add_summary(summary_str, epoch)
        
        #save at every 5% of total epochs number - so 21 saves total.
        if epoch % int(n_epochs*(0.05))==0:
            saver.save(sess,checkpoint_epoch_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d"%(epoch+1))
                
    #at the end of the epochs, save the final model path.
    saver.save(sess, final_model_path)
    
    #now evaluate the probability of y_proba
    y_proba_val = y_proba.eval(feed_dict={X:X_test_N,y:y_test_N})